In [0]:
--MAC 결합
-- create table sandbox.z_luckyminki_choi.heds_1288_1 as (
select 
  t1.*,
  sha2(CONCAT(t2.mac_address, secret('admin', 'salt')), 256) as mac_addr
from sandbox.z_luckyminki_choi.heds_1288 t1 
left outer join kic_data_private.tdm.tb_manufacture t2
on t1.serial_no = t2.set_serial_no
-- )
;

-- 해당 mac들의 normal log 분리
-- create table sandbox.z_luckyminki_choi.heds_1288_1_normal as (
insert into sandbox.z_luckyminki_choi.heds_1288_1_normal 
with raw as (
  select 
    serial_no,
    tool,
    platform_version,
    before_after,
    yyyymmdd,
    mac_addr
  from sandbox.z_luckyminki_choi.heds_1288_1
), mac as (
  select distinct platform_version, mac_addr
  from raw
), mac22 as (
  select *
  from mac
  where platform_version in ('webOS22','ODM')
), mac23 as (
  select *
  from mac
  where platform_version in ('webOS23','ODM')
), mac24 as (
  select *
  from mac
  where platform_version in ('webOS24','ODM')
), mac25 as (
  select *
  from mac
  where platform_version in ('webOS25','ODM')
), normal_log as (
  select mac_addr, normal_log, context_name, message_id, log_create_time
  from kic_data_ods.tlamp.normal_log_webos22
  where 1=1
  and mac_addr in (select mac_addr from mac22)
  and ((context_name = 'com.webos.app.quicksettings' and message_id = 'NL_QUICKSETTINGS_VALUE')
    OR (context_name = 'pqcontroller' and message_id in ('NL_PICTURE_PERIODIC_REPORT','NL_PICTURE_POWER_ON_REPORT'))
    OR (context_name = 'umi' 
        and message_id = 'NL_SOUND' 
        and (normal_log:volume is not null OR normal_log:inputaudioformathdmi1 is not null OR normal_log:avSync is not null)))
)
select
  serial_no,
  tool,
  platform_version,
  before_after,
  yyyymmdd,
  raw.mac_addr,
  normal_log.log_create_time,
  normal_log.context_name, 
  normal_log.message_id,
  normal_log.normal_log
from raw
inner join normal_log
on raw.mac_addr=normal_log.mac_addr
-- )

;



-- 최종 output 형태로 조회
with raw as (
  select 
  serial_no,
  platform_version,
  context_name,
  message_id,
  normal_log,
  log_create_time
  from sandbox.z_luckyminki_choi.heds_1288_1_normal
  where 1=1
  and before_after = '후'
  and log_create_time > yyyymmdd
), nl_quicksetting_picture_button as (
  select 
    serial_no, 
    platform_version, 
    normal_log:quicksettings_value as QuickSettings_picture_button,
    row_number() over(partition by serial_no, normal_log:quicksettings_item order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'com.webos.app.quicksettings'
  and message_id = 'NL_QUICKSETTINGS_VALUE'
  and normal_log:quicksettings_item = 'QuickSettings_picture_button' 
), nl_quicksetting_soundmode_button as (
  select 
    serial_no, 
    platform_version, 
    normal_log:quicksettings_value as QuickSettings_soundMode_button,
    row_number() over(partition by serial_no, normal_log:quicksettings_item order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'com.webos.app.quicksettings'
  and message_id = 'NL_QUICKSETTINGS_VALUE'
  and normal_log:quicksettings_item = 'QuickSettings_soundMode_button' 
), nl_quicksetting_soundout_button as (
  select 
    serial_no, 
    platform_version,
    normal_log:quicksettings_value as QuickSettings_soundOut_button,
    row_number() over(partition by serial_no, normal_log:quicksettings_item order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'com.webos.app.quicksettings'
  and message_id = 'NL_QUICKSETTINGS_VALUE'
  and normal_log:quicksettings_item = 'QuickSettings_soundOut_button' 
), nl_picture_periodic_report as (
  select 
    serial_no, 
    platform_version, 
    normal_log:energy_saving as energy_saving,
    normal_log:backlight as backlight,
    normal_log:brightness as brightness,
    normal_log:picture_mode as picture_mode,
    row_number() over(partition by serial_no order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'pqcontroller'
  and message_id ='NL_PICTURE_PERIODIC_REPORT'
  and normal_log:picture_mode is not null
), nl_picture_power_on_report as (
  select 
    serial_no, 
    platform_version, 
    normal_log:ai_picture as ai_picture,
    normal_log:ai_brightness as ai_brightness,
    normal_log:ai_genre as ai_genre,
    row_number() over(partition by serial_no order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'pqcontroller'
  and message_id ='NL_PICTURE_POWER_ON_REPORT'
  and normal_log:picture_mode is not null
), nl_sound1 as (
  select 
    serial_no, 
    platform_version, 
    normal_log:soundMode as soundMode,
    normal_log:soundOutput as soundOutput,
    normal_log:supportATMOS as supportATMOS,
    normal_log:tvSetupConfiguration as tvSetupConfiguration,
    normal_log:magicSpaceSound as magicSpaceSound,
    normal_log:volume as `volume`,
    normal_log:autoVolume as autoVolume,
    normal_log:soundOutputDigital as soundOutputDigital,
    normal_log:eArcSupport as eArcSupport,
    row_number() over(partition by serial_no order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'umi'
  and message_id ='NL_SOUND'
  and normal_log:soundMode is not null
), nl_sound2 as (
  select 
    serial_no, 
    platform_version, 
    normal_log:avSync as avSync,
    normal_log:avSyncSpeaker as avSyncSpeaker,
    normal_log:avSyncSpdif as avSyncSpdif,
    normal_log:avSyncWisa as avSyncWisa,
    normal_log:avSyncBtsurround as avSyncBtsurround,
    normal_log:avSyncBypassInput as avSyncBypassInput,
    row_number() over(partition by serial_no order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'umi'
  and message_id ='NL_SOUND'
  and normal_log:avSync is not null
), nl_sound3 as (
  select 
    serial_no, 
    platform_version, 
    normal_log:inputAudioFormatHDMI1 as inputAudioFormatHDMI1,
    normal_log:inputAudioFormatHDMI2 as inputAudioFormatHDMI2,
    normal_log:inputAudioFormatHDMI3 as inputAudioFormatHDMI3,
    normal_log:inputAudioFormatHDMI4 as inputAudioFormatHDMI4,
    row_number() over(partition by serial_no order by log_create_time desc) as rk
  from raw
  where 1=1
  and context_name = 'umi'
  and message_id ='NL_SOUND'
  and normal_log:inputAudioFormatHDMI1 is not null
)
select 
  bsic.serial_no,
  bsic.tool,
  bsic.platform_version,
  bsic.before_after,
  bsic.yyyymmdd,
  nvl(nl_quicksetting_picture_button.QuickSettings_picture_button,'') as QuickSettings_picture_button,
  nvl(nl_quicksetting_soundmode_button.QuickSettings_soundMode_button,'') as QuickSettings_soundMode_button,
  nvl(nl_quicksetting_soundout_button.QuickSettings_soundOut_button,'') as QuickSettings_soundOut_button,
  nvl(nl_picture_periodic_report.energy_saving,'') as energy_saving,
  nvl(nl_picture_periodic_report.backlight,'') as backlight,
  nvl(nl_picture_periodic_report.brightness,'') as brightness,
  nvl(nl_picture_periodic_report.picture_mode,'') as picture_mode,
  nvl(nl_picture_power_on_report.ai_picture,'') as ai_picture,
  nvl(nl_picture_power_on_report.ai_brightness,'') as ai_brightness,
  nvl(nl_picture_power_on_report.ai_genre,'') as ai_genre,
  nvl(nl_sound1.soundmode,'') as soundMode,
  nvl(nl_sound1.soundOutput,'') as soundOutput,
  nvl(nl_sound1.supportATMOS,'') as supportATMOS,
  nvl(nl_sound1.tvSetupConfiguration,'') as tvSetupConfiguration,
  nvl(nl_sound1.magicSpaceSound,'') as magicSpaceSound,
  nvl(nl_sound1.volume,'') as volume,
  nvl(nl_sound1.autoVolume,'') as autoVolume,
  nvl(nl_sound1.soundOutputDigital,'') as soundOutputDigital,
  nvl(nl_sound1.eArcSupport,'') as eArcSupport,
  nvl(nl_sound2.avSync,'') as avSync,
  nvl(nl_sound2.avSyncSpeaker,'') as avSyncSpeaker,
  nvl(nl_sound2.avSyncSpdif,'') as avSyncSpdif,
  nvl(nl_sound2.avSyncWisa,'') as avSyncWisa,
  nvl(nl_sound2.avSyncBtsurround,'') as avSyncBtsurround,
  nvl(nl_sound2.avSyncBypassInput,'') as avSyncBypassInput,
  nvl(nl_sound3.inputaudioformathdmi1,'') as inputaudioformathdmi1,
  nvl(nl_sound3.inputaudioformathdmi2,'') as inputaudioformathdmi2,
  nvl(nl_sound3.inputaudioformathdmi3,'') as inputaudioformathdmi3,
  nvl(nl_sound3.inputaudioformathdmi4,'') as inputaudioformathdmi4
from (
  select serial_no,tool,platform_version,before_after,yyyymmdd
  from sandbox.z_luckyminki_choi.heds_1288_1
  where 1=1
  and before_after = '후'
) bsic
left outer join nl_quicksetting_picture_button
on bsic.serial_no=nl_quicksetting_picture_button.serial_no
and bsic.platform_version=nl_quicksetting_picture_button.platform_version
and nl_quicksetting_picture_button.rk=1
left outer join nl_quicksetting_soundmode_button
on bsic.serial_no=nl_quicksetting_soundmode_button.serial_no
and bsic.platform_version=nl_quicksetting_soundmode_button.platform_version
and nl_quicksetting_soundmode_button.rk=1
left outer join nl_quicksetting_soundout_button
on bsic.serial_no=nl_quicksetting_soundout_button.serial_no
and bsic.platform_version=nl_quicksetting_soundout_button.platform_version
and nl_quicksetting_soundout_button.rk=1
left outer join nl_picture_periodic_report
on bsic.serial_no=nl_picture_periodic_report.serial_no
and bsic.platform_version=nl_picture_periodic_report.platform_version
and nl_picture_periodic_report.rk=1
left outer join nl_picture_power_on_report
on bsic.serial_no=nl_picture_power_on_report.serial_no
and bsic.platform_version=nl_picture_power_on_report.platform_version
and nl_picture_power_on_report.rk=1
left outer join nl_sound1
on bsic.serial_no=nl_sound1.serial_no
and bsic.platform_version=nl_sound1.platform_version
and nl_sound1.rk=1
left outer join nl_sound2
on bsic.serial_no=nl_sound2.serial_no
and bsic.platform_version=nl_sound2.platform_version
and nl_sound2.rk=1
left outer join nl_sound3
on bsic.serial_no=nl_sound3.serial_no
and bsic.platform_version=nl_sound3.platform_version
and nl_sound3.rk=1
;

